In [10]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [11]:
# from rs import skill_check

# def test_skill_check(metadata):
#     print(skill_check("sanity", None, metadata))

# metadata = {
#     "character": {"sanity":30},
#     "symptom":[
#         "incoherent speech",
#         "uncontrollable twitching, trembling",
#         "delusions of persecution",
#         "strange appetites (dirt, clay, etc...)",
#         "scratches, punches, and bruises all over the body"
#     ]
# }

# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)

# print(metadata["character"])


In [12]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|normal|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    
    # print(config.metadata["character"])
    
    if "tags" in config.metadata["character"] and "insane" in config.metadata["character"]["tags"]:
        return f"TURN: {count + 1} {is_final} (I'm insane!)"
    
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    # print(config.metadata["character"])
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        # config.metadata["memories"] = []
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/ai_gf.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

The year is 2023, you travelled to some distant mountains and forest in Hokkaido to do some scenic photography in the middle of the week during autumn. You wanted to get away from the city life of Tokyo just so you can make some time for yourself and think about the future and want to do in your life. As you are hiking through the quiet mountain pass, you hear a cry of a girl just in the distance. You quickly made your way towards the sound and found a beautiful young lady on the side of the road, and appears to have accidently injured herself.

What is your response?

possible actions:
1. Introduce yourself and offer assitance
2. Say a funny pick up line
3. Ignore her and keep walking


I select option 1. TURN: 1 


<start>
As you approach the young lady, you notice her trying to stand up, wincing in pain. You introduce yourself, "Hi, I'm Hikaru Mikami, a photographer. I heard your cry for help. Are you okay? Can I assist you in any way?"

Mitsuha looks up at you, her big brown eyes reflecting a mix of surprise and relief. "Oh, hello... I'm Mitsuha Tomoe. I was trying to take a shortcut and I think I twisted my ankle. I'm not sure I can walk well on it," she responds, her voice soft and timid, yet there's a hint of cheekiness as she tries to lighten the situation despite her evident discomfort.

You notice her attempting to stand again, clearly struggling.

possible actions:
1. Offer to help her stand and see if she can put weight on her ankle.
2. Suggest calling for help since she might need professional medical attention.
3. Offer to carry her to a safer location or back to her hotel if it's nearby.
<end>

I select option 1. TURN: 2 


<start>
You carefully offer your hand to assist Mitsuha, saying, "Let's see if you can stand on it, but please be careful. We don't want to make it worse." With your support, she slowly gets up, trying to put a little weight on her injured ankle. She winces slightly but manages a small, grateful smile.

"It hurts, but I think I can manage a few steps with some help," Mitsuha admits, leaning on you for support. Her cheekiness fades a bit, replaced by a look of determination mixed with a touch of vulnerability.

Seeing her in pain but trying to stay strong sparks a protective instinct in you.

possible actions:
1. Suggest taking a short break to rest her ankle before deciding the next step.
2. Offer to support her as she tries to walk a little.
3. Insist on carrying her to avoid further injury, as walking might not be the best idea.
<end>

I select option 3. TURN: 3 


<start>
You look at Mitsuha with a determined expression and say, "I think it's best if I carry you. We don't want to risk further injury by putting more strain on your ankle." Before she can protest, you gently lift her in a bridal style, making sure she's comfortable and not in any pain.

Mitsuha's cheeks turn a shade rosier, possibly from the unexpected closeness or the surprise at your decisive action. She wraps her arms lightly around your neck, trying not to impose too much. "Thank you, Hikaru. I'm sorry for the trouble," she says, her voice soft and filled with genuine appreciation.

You assure her it's no trouble at all and ask her where she's staying.

She tells you about a small hotel in the mountains, not too far from where you found her. It seems she was quite adventurous to wander off the beaten path.

possible actions:
1. Start heading towards the hotel she mentioned, making small talk to keep her mind off the pain.
2. Ask her more about what

I select option 3. TURN: 4 


<start>
As you carefully navigate the path towards the hotel, you decide to share a bit about yourself to make Mitsuha feel more at ease. "I'm actually here in Hokkaido for a bit of a personal retreat. I'm a photographer, so I was hoping to capture the autumn scenery. It's a way for me to clear my head and think about the future," you explain, your voice calm and reflective.

Mitsuha listens intently, her gaze fixed on you. "That sounds wonderful. I've always admired people who can express themselves through art. It must be nice to connect with nature and find inspiration in its beauty," she responds, her voice carrying a note of admiration and a hint of longing. It's clear your passion for photography has piqued her interest, and she seems to momentarily forget about her injury.

The conversation flows easily between you two, creating a comfortable atmosphere as you continue your journey to the hotel.

possible actions:
1. Encourage Mitsuha to share more 

I select option 1. TURN: 5 


<start>
Encouraged by the easy flow of conversation, you gently prompt Mitsuha to share more about herself. "I'd love to hear more about you, Mitsuha. What brings you to the mountains? Any particular interests or hobbies you're passionate about?" you ask, genuinely interested in knowing more about her.

Mitsuha seems to brighten at the question, a soft enthusiasm lighting up her eyes. "Well, I'm actually here on a short vacation. I work in Tokyo, and sometimes it just gets a bit overwhelming. I love nature, and I've always found the mountains to be particularly calming. As for hobbies, I enjoy reading and I've recently started trying my hand at sketching. It's nothing professional, just a way to capture moments that feel special to me," she shares, her voice warm and a bit more animated as she talks about her interests.

As you listen to Mitsuha, you find yourself admiring her more—not just for her physical appearance but for her inner world and the simple